In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import boto3
import re


import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig
from sagemaker.model_monitor import DataCaptureConfig, DatasetFormat, DefaultModelMonitor
from sagemaker.s3 import S3Uploader, S3Downloader

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

In [84]:
sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()

In [85]:
sess
sm
role

'arn:aws:iam::647453829825:role/service-role/AmazonSageMaker-ExecutionRole-20200729T003498'

In [86]:
account_id = sess.client('sts', region_name=sess.region_name).get_caller_identity()["Account"]
bucket = 'sagemaker-studio-{}-{}'.format(sess.region_name, account_id)
prefix = 'HF_Prop_cal'

In [87]:
print(account_id)
print(bucket)
print(prefix)

647453829825
sagemaker-studio-eu-west-1-647453829825
HF_Prop_cal


In [88]:
bucket = 'sagemaker-studio-i8kf84zy9f'

In [145]:
from sagemaker.amazon.amazon_estimator import get_image_uri
#docker_image_name = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='0.90-2')
docker_image_name = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='latest')

	get_image_uri(region, 'xgboost', '1.0-1').


In [90]:
s3_marker201 = sagemaker.s3_input(s3_data='s3://{}/{}/{}'.format(bucket, prefix,'201.csv'), content_type='csv')

In [143]:
docker_image_name

'685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'

In [92]:
s3 = boto3.client('s3')
s3.download_file('sagemaker-studio-i8kf84zy9f', 'HF_Prop_cal/201.csv', '_201.csv')

In [93]:
#initiate s3 resource
s3 = boto3.resource('s3')

# select bucket
my_bucket = s3.Bucket('sagemaker-studio-i8kf84zy9f')
def downloadDirectoryFroms3(bucketName, remoteDirectoryName):
    s3_resource = boto3.resource('s3')
    bucket = s3_resource.Bucket(bucketName) 
    for obj in bucket.objects.filter(Prefix = remoteDirectoryName):
        if (obj.key == remoteDirectoryName + "/"):
            continue
        if not os.path.exists(os.path.dirname(obj.key)):
            os.makedirs(os.path.dirname(obj.key))
        bucket.download_file(obj.key, obj.key)
        
downloadDirectoryFroms3('sagemaker-studio-i8kf84zy9f', 'HF_Prop_cal')
downloadDirectoryFroms3('sagemaker-studio-i8kf84zy9f', 'oil')

In [163]:
from importlib import reload # python 2.7 does not require this

from geo_config import oil_files_list
oil_files_list = ["oil/oil-0.csv",
                  "oil/oil-0(1).csv"]
import main_forest
reload(main_forest)
from main_forest import create_trainig_set
oil_files_list
X, y = create_trainig_set()

load oil markers...
Load oil data from:  ['oil/oil-0.csv', 'oil/oil-0(1).csv']
Total 454904 oil samples was loaded
Size of oil dataset was reduced to 200000 samples
Start train model for oil classification...
Finish train model for oil classification. Filtered data frame size:  (67147, 9)
['HF_Prop_cal/201.csv' 'HF_Prop_cal/202.csv' 'HF_Prop_cal/203.csv'
 'HF_Prop_cal/205.csv' 'HF_Prop_cal/206.csv' 'HF_Prop_cal/209.csv'
 'HF_Prop_cal/210.csv' 'HF_Prop_cal/212.csv' 'HF_Prop_cal/213.csv'
 'HF_Prop_cal/214.csv']
From file  HF_Prop_cal/201.csv  loaded dataframe of size:  (5115, 29)
After filtering marker data frame size:  (4797, 9)
From file  HF_Prop_cal/202.csv  loaded dataframe of size:  (2336, 29)
After filtering marker data frame size:  (2177, 9)
From file  HF_Prop_cal/203.csv  loaded dataframe of size:  (9533, 29)
After filtering marker data frame size:  (8905, 9)
From file  HF_Prop_cal/205.csv  loaded dataframe of size:  (2750, 29)
After filtering marker data frame size:  (2612, 9)
F

In [ ]:
print(X)
print(y)
print(len(X), len(y))

In [288]:
y_T = np.transpose([y])
data1 = np.empty((len(y), 0), dtype=np.object)
data1 = np.append(data1, y.reshape(-1, 1), axis=1) 
data1 = np.append(data1, X, axis=1) 
#data = np.concatenate((y_T, X), axis=1)
data = data1
np.random.shuffle(data)
#print(data)
train_data, validation_data, test_data = np.split(data, [int(0.7 * len(data)), int(0.9 * len(data))])
df = pd.DataFrame(data)

train_data, validation_data, test_data = np.split(df.sample(frac=1, random_state=1729), [int(0.7 * len(df)), int(0.9 * len(df))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)
test_data.to_csv('test.csv', header=False, index=False)


# np.savetxt("train.csv", train_data, delimiter=",")
# np.savetxt("validation.csv", validation_data, delimiter=",")
# np.savetxt("test.csv", test_data, delimiter=",")

In [289]:
data1[:5,:]

array([[2, 6.5792895915075595, 6.356858635796325, 5.834568198866494,
        2.45178643552429, 2.456366033129043, 4.112102054770891,
        5.736058497640671, 4.388190041599854, 4.164709663539879],
       [10, 3.119585774961784, 2.816241299991783, 2.250420002308894,
        1.863322860120456, 1.9637878273455553, 4.103393169973528,
        3.3604040547299387, 2.0492180226701815, 1.5440680443502757],
       [5, 6.225109667203757, 6.587406403231895, 5.981935882545361,
        3.074450718954591, 3.058426024457005, 5.907307074579468,
        5.532584958366993, 6.571627786992635, 6.258422955557263],
       [5, 6.407728689877719, 6.792883354657924, 6.18653086867832,
        3.0824263008607717, 3.064832219738574, 6.089202964360833,
        5.82727038224852, 6.724262676681238, 6.411457404368348],
       [10, 2.531478917042255, 3.036628895362161, 0.3010299956639812,
        2.05307844348342, 2.1702617153949575, 3.6009728956867484,
        2.8419848045901137, 1.3222192947339193, 1.81291335664285

In [290]:
prefix=''
s3_input_train = boto3.Session().resource('s3').Bucket('sagemaker-studio-eu-west-1-647453829825').Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_validation = boto3.Session().resource('s3').Bucket('sagemaker-studio-eu-west-1-647453829825').Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [110]:
create_date = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
markers_experiment = Experiment.create(experiment_name="geosplit-classification-{}".format(create_date), 
                                              description="Using xgboost to predict markers", 
                                              sagemaker_boto_client=boto3.client('sagemaker'))

In [ ]:

sess = sagemaker.Session()
prefix = ''
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/train'.format('sagemaker-studio-eu-west-1-647453829825'), content_type='csv') 
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/validation/'.format('sagemaker-studio-eu-west-1-647453829825'), content_type='csv')

# hyperparams = {"max_depth":5,
#                "subsample":0.8,
#                "num_round":600,
#                "eta":0.2,
#                "gamma":4,
#                "min_child_weight":6,
#                "silent":0,
#                "objective":'logistic'}
hyperparams = {"max_depth":5,
               "subsample":0.8,
               "num_round":25,
               "eta":0.2,
               "gamma":4,
               "min_child_weight":6,
               "silent":0,
               "num_class": 11,
               "objective": "multi:softprob"}#,
               #"eval_metric": "error"}
trial = Trial.create(trial_name="algorithm-mode-trial-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime())), 
                     experiment_name=markers_experiment.experiment_name,
                     sagemaker_boto_client=boto3.client('sagemaker'))
xgb = sagemaker.estimator.Estimator(image_name=docker_image_name,
                                    role=role,
                                    hyperparameters=hyperparams,
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/output'.format(bucket),
                                    base_job_name="demo-xgboost",
                                    sagemaker_session=sess)
#xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation},
       experiment_config={
            "ExperimentName": markers_experiment.experiment_name, 
            "TrialName": trial.trial_name,
            "TrialComponentDisplayName": "Training",
        })

In [292]:
data_capture_prefix = '{}/datacapture'.format('xgboost-geosplit')
endpoint_name = "demo-xgboost-geosplit-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName = {}".format(endpoint_name))

EndpointName = demo-xgboost-geosplit-2020-08-07-19-31-39


In [170]:
sess.delete_endpoint(xgb_predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: demo-xgboost-geosplit-2020-07-31-15-35-22


In [293]:

xgb_predictor = xgb.deploy(initial_instance_count=1, 
                           instance_type='ml.m4.xlarge',
                           endpoint_name=endpoint_name,
                           data_capture_config=DataCaptureConfig(enable_capture=True,
                                                                 sampling_percentage=100,
                                                                 destination_s3_uri='s3://{}/{}'.format(bucket, data_capture_prefix)
                                                                )
                           )

INFO:sagemaker:Creating model with name: demo-xgboost-2020-08-07-19-21-40-607
INFO:sagemaker:Creating endpoint with name demo-xgboost-geosplit-2020-08-07-19-31-39


-------------!

In [296]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

In [ ]:
def predict(data, rows=1):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.to_numpy()[:1, 1:])
print(len(predictions))

In [313]:
k = 4

td = test_data.to_numpy()
print(td[0, 1:])
pred = xgb_predictor.predict(td[:4, 1:])
print("pred = ", pred)
raw_pred = pred[1:-1].replace(b'],[', b', ')

pred_all = np.fromstring(raw_pred, sep=',').reshape(k, 11)
print(pred_all)
print(pred_all.shape)
print(train_data.shape)
print(test_data.shape)

[5.08181879785487 4.411888952395849 3.754501229386917 1.9344984512435677
 2.187520720836463 5.995385952115371 6.413065953467973 2.6757783416740852
 2.6848453616444123]
pred =  b'[0.0018283948302268982, 0.9893065094947815, 0.000868268427439034, 0.0008486137958243489, 0.000848134804982692, 0.0008490047766827047, 0.0008541177958250046, 0.001368816359899938, 0.0008498586248606443, 0.0008519701077602804, 0.001526355859823525],[0.0009593224385753274, 0.0009361457778140903, 0.0009492176468484104, 0.0009157682070508599, 0.0009152504499070346, 0.000996708287857473, 0.0009217077749781311, 0.0009576377342455089, 0.9905390739440918, 0.0009193900623358786, 0.0009897802956402302],[0.0008750646375119686, 0.0008761147037148476, 0.0008825373952277005, 0.0008625597110949457, 0.0008620720473118126, 0.00086295633809641, 0.0008681542240083218, 0.0008730169502086937, 0.0008638245635665953, 0.0008659712038934231, 0.99130779504776],[0.0009665871621109545, 0.0009677475318312645, 0.0009748410666361451, 0.989883

In [218]:
ind = np.fromstring(pred, sep=',')
from main_forest import load_training_set
from geo_config import training_set_path

int_ind = ind.astype(int)
print(ind, int_ind)
filenames, marker_names = load_training_set(training_set_path)
marker_names = np.append(marker_names, 'oil')
print(marker_names)
print(marker_names[int_ind[0]])

[10. 10.  4. ... 10. 10.  0.] [10 10  4 ... 10 10  0]
['marker201' 'marker202' 'marker203' 'marker205' 'marker206' 'marker209'
 'marker210' 'marker212' 'marker213' 'marker214' 'oil']
oil


In [235]:
from sklearn import metrics
score = metrics.accuracy_score(test_data.to_numpy()[:, 0], pred)
print(score)

0.9998132411989915
[10. 10.  4. 10. 10. 10. 10. 10.]


In [238]:
def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=',', fmt='%g')
    return csv.getvalue().decode().rstrip()

In [ ]:
runtime= boto3.client('runtime.sagemaker')
payload = np2csv(td[:, 1:])
response = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType='text/csv',
                                   Body=payload)
print(response['Body'].read().decode())

In [268]:
from sagemaker import PCA
num_components=8

pca_SM = PCA(role=role,
             train_instance_count=1,
             train_instance_type='ml.c4.xlarge',
             output_path='s3://'+ bucket +'/pca/',
             num_components=num_components)

In [270]:
from geo_config import features_to_train
train_data2 = train_data.values.astype('float32')[:,1:]
print(features_to_train)
train_data2.shape


['BL-530-H', 'BL-586-H', 'BL-615-H', 'RL-660-H', 'RL-695-H', 'VL-445-H', 'VL-530-H', 'YL-586-H', 'YL-615-H']


(74960, 9)

In [ ]:
%%time
pca_SM.fit(pca_SM.record_set(train_data2))

In [272]:
job_name = pca_SM.latest_training_job.name
model_key = "pca/" + job_name + "/output/model.tar.gz"

boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')

0

In [273]:
import mxnet as mx
pca_model_params = mx.ndarray.load('model_algo-1')

ModuleNotFoundError: No module named 'mxnet'

In [278]:
!pip install mxnet

     |████████████████████████████████| 68.7 MB 108 kB/s  eta 0:00:01


In [279]:
import mxnet as mx
pca_model_params = mx.ndarray.load('model_algo-1')

In [280]:
s=pd.DataFrame(pca_model_params['s'].asnumpy())
v=pd.DataFrame(pca_model_params['v'].asnumpy())

In [287]:
s.iloc[4:,:].apply(lambda x: x*x).sum()/s.apply(lambda x: x*x).sum()

0    0.985834
dtype: float32